In [25]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
import json
import requests
import pandas as pd
import numpy

# Load environment variables from the .env file
load_dotenv()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 


In [20]:
# Database file path
database_file_path = "./db/test.db"

# Create the SQLAlchemy engine
engine = create_engine(f'sqlite:///{database_file_path}')

@tool
def calculate_price(product_list: list) -> float:
    """
    Calculates the total price of a list of products.

    Args:
        product_list (list): A list of dictionaries representing products.
        
    Returns:
        float: The total price of the products.
    """
    total_price = sum(product['price'] for product in product_list)
    return total_price

@tool
def get_product_price(product_name: str) -> str:
    """
    Retrieves the price of a specific product.

    Args:
        product_name (str): The name of the product.
        
    Returns:
        str: The price of the product in JSON format.
    """
    try:
        query = text("""
            SELECT name, price
            FROM Product
            WHERE name LIKE :product_name;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection, params={"product_name": f"%{product_name}%"})
        
        if not result.empty:
            return json.dumps(result.to_dict('records')[0])
        else:
            return json.dumps({"product_name": product_name, "price": None})
    except Exception as e:
        return json.dumps({"error": str(e)})

@tool
def get_products_available() -> str:
    """
    Retrieves the list of products that are available.

    Returns:
        str: A list of available products in JSON format.
    """
    try: 
        query = text("""
            SELECT name, price
            FROM Product
            WHERE available = 1;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        
        if not result.empty:
            return json.dumps(result.to_dict('records'))
        else:
            return json.dumps({"available_products": []})
    except Exception as e:
        return json.dumps({"error": str(e)})

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Define the tools
tools_sql = [get_product_price, get_products_available, calculate_price]

# Get the API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Create the agent
agent = create_tool_calling_agent(llm, tools_sql, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_sql, verbose=True)

# Example usage
# agent_executor.invoke({"input": "What is the total cost for a portable charger and a keyboard?"})


## Tools / API calls

### Token

In [81]:
base_url = 'https://dashboard.colorines.paitesting.com/api'

# Set the URL for the token API
url_token = base_url+"/v1/token"

# Set the data for the POST request
data = {
    "email": "sale@platform.com",
    "password": os.getenv('CONSULTANT_PASSWORD'),
    "device_name": "colorina_2",
    "role": "consultant"
}

# Send a POST request
response = requests.post(url_token, json=data)

#print(response)

# Extract the token from the response
token = response.json().get('data').get('token')

#print("Token:", token)


## Components

In [137]:
import requests
import pandas as pd
import json

#TODO: API need to add a filter by 'is_active'

def GetProperties(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/properties"
    params = {'include': 'step,project,development'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the response contains any content
        if response.content:
            data = response.json()
        else:
            return pd.DataFrame({"error": ["Empty response from the server"]})
        
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    except ValueError as ve:
        return pd.DataFrame({"error": ["Error parsing JSON response"]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})
    
#PREPROCESS projects
def PreprocessProjects(df):
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    
    def filter_active_steps(steps):
        return [step['step_number'] for step in steps if step['is_active']]

    # Applying the function to create the new column
    df['active_steps'] = df['steps'].apply(filter_active_steps)

    return df

#PREPROCESS Properties
def PreprocessProperties(df):
    df['project_name'] = df['project'].apply(lambda x: x.get('name'))
    df['development_name'] = df['development'].apply(lambda x: x.get('name'))
    df['step_number'] = df['step'].apply(lambda x: x.get('step_number')) #numero de etapa

    def extract_total_amount(prices, currency_code):
        total_amount = None  # Initialize to None or any default value as needed
        for entry in prices:
            if entry['type'] == 'property' and entry['currency_code'] == currency_code:
                total_amount = entry['total_amount']
                break  # Exit loop once found
        return total_amount

    # Extracting total_amount_mxn and total_amount_usd
    df['total_amount_mxn'] = df['prices'].apply(lambda x: extract_total_amount(x, 'MXN'))
    df['total_amount_usd'] = df['prices'].apply(lambda x: extract_total_amount(x, 'USD'))

    return df


# Example usage

#df_properties = GetProperties(token)

def GetProjects(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/projects"
    params = {'include': 'steps,development', 'is_active': 'True'} 
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

df_projects = GetProjects(token)

def GetDevelopments(token: str):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/developments"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        return pd.DataFrame({"error": [str(e)]})
    
    # Check if the 'data' key exists and contains a list of dictionaries
    if 'data' in data:
        if isinstance(data['data'], list):
            return pd.DataFrame(data['data'])
        else:
            return pd.DataFrame({"error": ["'data' key does not contain a list"]})
    else:
        return pd.DataFrame({"error": ["'data' key not found in the response"]})

# Example usage

#df_devs = GetDevelopments(token)



## Clean functions

In [134]:
def CleanProjects(df):
    df_clean = df[['name', 'development_name', 'active_steps']]
    df_clean = df_clean.rename(columns={'name': 'nombre_proyecto', 'development_name': 'nombre_desarrollo', 'active_steps': 'etapa'})
    return df_clean.to_json(orient='records', force_ascii=False)

In [28]:
properties_df = df_properties[df_properties['is_active'] == True]

## Tools

In [125]:
from langchain_core.tools import tool
import requests
import json

import requests
import json

@tool
def get_active_properties(token: str) -> str:
    """
    Uses the token to fetch the active properties from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    properties_df = GetProperties(token)
    properties_df = properties_df[properties_df['is_active'] == True]
    

    response = requests.get(url, headers=headers, params=params)
    # Check if the response was successful
    if response.status_code == 200:
        try:
            return json.dumps(response.json())  # Convert JSON response to string
        except ValueError:  # Catch JSON errors
            return json.dumps({"error": "Error parsing JSON"})
    else:
        return json.dumps({"error": f"Received status code {response.status_code}"})



@tool
def get_last_sale(token: str) -> str:
    """
    Uses the token to fetch the last sale from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
    params = {"per_page": 1, "sort": "-created_at"}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})



### Properties, developments and Steps

In [123]:
@tool
def get_active_projects(token: str) -> str:   
    """
    Uses the token to fetch the active projects from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    df = GetProjects(token)
    pre_df = PreprocessProjects(df)
    return CleanProjects(pre_df)
    
    

## Test agent

In [135]:
    
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who will use the {token} to use the tools to bring information to the user"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Example of integrating with langchain agent framework
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Add the `get_sales` function to the list of tools
tools_api = [get_last_sale, get_active_projects] 

# Create the agent
agent = create_tool_calling_agent(llm, tools_api, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_api, verbose=True)

# Example usage:
# token = 'your_actual_token_here'
# base_url = 'https://api.yourdomain.com'


In [138]:
get_active_projects(token)

'[{"nombre_proyecto":"Tres Raíces","nombre_desarrollo":"Yucatán","etapa":[4]}]'

In [136]:
input_data = {
    "input": "dime los proyectos activos",
    "token": token
}

# Invoke the agent
agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...

Invoking: `get_active_projects` with `{'token': '2116|2Y18BZCfbUvOJDipbWQgv0BdGnimrk4EC62q8rOY11db864b'}`


[{"nombre_proyecto":"Tres Raíces","nombre_desarrollo":"Yucatán","etapa":[4]}]El proyecto activo actualmente es "Tres Raíces" en el desarrollo de Yucatán, en la etapa 4.

> Finished chain.


{'input': 'dime los proyectos activos',
 'token': '2116|2Y18BZCfbUvOJDipbWQgv0BdGnimrk4EC62q8rOY11db864b',
 'output': 'El proyecto activo actualmente es "Tres Raíces" en el desarrollo de Yucatán, en la etapa 4.'}

## Sales

In [ ]:
@tool
def get_customers(base_url: str, token: str) -> str:
    """
    Fetches a list of customers from Colorines API endpoint.

    Args:
    - base_url (str): The base URL of the API.
    - token (str): The authorization token required to access the API.

    Returns:
    - str: The JSON response from the API as a string.
    """
    headers = {
        "Authorization": f"Bearer {token}"
    }
    url_list_customers = f"{base_url}/v1/customers"
    response = requests.get(url_list_customers, headers=headers)
    return response.json()

# Example usage:
customers_data = get_customers(base_url, token)
customers_data